# Making HTTP Requests

In Python, there are many libraries to make HTTP requests. We will use a 3rd-party library called "requests", which is very easy to use. 

Making a "GET" request is as simple as: 

```python
import requests

res = requests.get(url) # returns a "Response" object
res.content # has the "body" of the response
```

You might need to install the requests library! 

You can do that with the following code in a Jupyter cell: 

```python
! pip install requests
```

Or, if you're using anaconda, optionally you can also do: 

```python
! conda install -c anaconda requests
```

## Parsing JSON data

To parse JSON data in Python, we will use the "json" module: 

```python
import json
```

Read more about the module on the [documentation page](https://docs.python.org/3/library/json.html)!

All we care about for this part is the method "loads", which turns JSON data into a Python object: 

```python
json.loads(my_string_encoded_json)
```

## Pokemon API

There is a simple, open API called "pokeapi" that allows us to make requests and see how to use APIs. Like everything, we first look at the documentation: 

https://pokeapi.co/docs/v2.html

In [28]:
# Let's see how to make a get request to the API: 
import requests
import json
import pandas as pd

res = requests.get('https://pokeapi.co/api/v2/pokemon?offset=20&limit=20')
js = json.loads(res.content)
js['next']

'https://pokeapi.co/api/v2/pokemon?offset=40&limit=20'

In [34]:
res = requests.get('https://pokeapi.co/api/v2/berry')
json_=json.loads(res.content)
json_

{'count': 64,
 'next': 'https://pokeapi.co/api/v2/berry?offset=20&limit=20',
 'previous': None,
 'results': [{'name': 'cheri', 'url': 'https://pokeapi.co/api/v2/berry/1/'},
  {'name': 'chesto', 'url': 'https://pokeapi.co/api/v2/berry/2/'},
  {'name': 'pecha', 'url': 'https://pokeapi.co/api/v2/berry/3/'},
  {'name': 'rawst', 'url': 'https://pokeapi.co/api/v2/berry/4/'},
  {'name': 'aspear', 'url': 'https://pokeapi.co/api/v2/berry/5/'},
  {'name': 'leppa', 'url': 'https://pokeapi.co/api/v2/berry/6/'},
  {'name': 'oran', 'url': 'https://pokeapi.co/api/v2/berry/7/'},
  {'name': 'persim', 'url': 'https://pokeapi.co/api/v2/berry/8/'},
  {'name': 'lum', 'url': 'https://pokeapi.co/api/v2/berry/9/'},
  {'name': 'sitrus', 'url': 'https://pokeapi.co/api/v2/berry/10/'},
  {'name': 'figy', 'url': 'https://pokeapi.co/api/v2/berry/11/'},
  {'name': 'wiki', 'url': 'https://pokeapi.co/api/v2/berry/12/'},
  {'name': 'mago', 'url': 'https://pokeapi.co/api/v2/berry/13/'},
  {'name': 'aguav', 'url': 'https

In [38]:
# Challenge: 
# Create a Dataframe with all the Pokemon names and their URLs. 

def get_pokes(url):
    res = requests.get(url)
    json_res = json.loads(res.content)

    next_url = json_res['next']
    # Make the HTTP request to the given url. 
    # Parse the response as json
    # return the "next" and the "results" (as a 2-tuple!)
    # make sure to return a "falsey" value (such as None)
    # if there is not a "next!"
    return next_url, json_res['results']


def catch_em_all(url):
    pokes = []
    
    # While loop! Like a for-loop, 
    # but goes on for an indetermined amount
    # of time:
    while url:
        url, results = get_pokes(url)
        pokes += results
    return pokes
        
    
list_of_pokes = catch_em_all('https://pokeapi.co/api/v2/pokemon')

# This data is most naturally represented as a list of dictionaries. 
# How can we create a dataframe from a list of dictionaries? 
# Try to find out on your own, from the internet!

# TODO: turn list_of_pokes into a dataframe.
df = pd.DataFrame(list_of_pokes)

In [39]:
df

,name,url
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/
3,charmander,https://pokeapi.co/api/v2/pokemon/4/
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/
...,...,...
959,araquanid-totem,https://pokeapi.co/api/v2/pokemon/10153/
960,togedemaru-totem,https://pokeapi.co/api/v2/pokemon/10154/
961,necrozma-dusk,https://pokeapi.co/api/v2/pokemon/10155/
962,necrozma-dawn,https://pokeapi.co/api/v2/pokemon/10156/


## Project: Live Exchange Rates

Imagine that you work with financial assets which are denominated in different currencies. You analyze this data regularly, and want to create a "transformation" function that transforms all your assets into EUR prices, based on today's exchange rate. 

Your data with the local-currency-denominated value of each asset lives in a file called "assets.csv" which should be located in the same folder as this notebook. 

Write a "data loading" function that: 

1. Reads the data, given the path to the file. 
2. Returns a dataframe with an additional column that has the assets value in euros, as of today.

Use this free API to get today's exchange rates: https://exchangeratesapi.io/. You will need to read the documentation and try it out to see how it works. 

HINT: Write a separate function to get the current exchange rates! That can be reused!

In [120]:
def read_data(path,base='EUR'):
    res = requests.get('https://api.exchangeratesapi.io/latest?base={}'.format(base))
    result = json.loads(res.content)
    df_rates=pd.DataFrame.from_dict(result['rates'], orient='index')
    df = pd.read_csv(path)
    df=df.merge(df_rates, how='left',left_on='curr', right_index=True).rename(columns={0: "exchange_rate"})
    df['val_in_{}'.format(base)]=df['value']/df['exchange_rate']
    return df.drop(columns=['exchange_rate'])

In [121]:
read_data(path='assets.csv',base='EUR')

,value,curr,val_in_EUR
0,48.910052,THB,1.460525
1,16.505115,THB,0.492867
2,30.370579,INR,0.387205
3,14.126967,SEK,1.314100
4,23.406904,HKD,2.692832
...,...,...,...
995,13.593894,HRK,1.823631
996,41.710860,ZAR,2.587490
997,12.877760,AUD,7.927215
998,29.561696,KRW,0.022791
